# Simple AI Workflow with RAG using OpenRouter and Supabase
#### Authored by Dr. Tiziana Ligorio for *AI Agents - CSCI 395.32* taught at Hunter College of The City University of New York

In this demo, we build a simple AI workflow that demonstrates the **Retrieval Augmented Generation (RAG)** pattern — a technique that grounds LLM responses in specific documentation, reducing hallucinations and enabling the model to answer questions about information not in its training data.

We will create a **FastHTML tutor** that can answer questions about the FastHTML library by retrieving relevant information from its documentation.

Our workflow consists of the following stages:

1. **Document Loading** — Load the FastHTML documentation from a text file
2. **Chunking** — Split the documentation into manageable pieces
3. **Embedding** — Create vector embeddings for each chunk using OpenAI embeddings
4. **Storage** — Store the embeddings in a Supabase vector database (Postgres + pgvector)
5. **Retrieval** — Query the database for relevant chunks based on user questions
6. **Generation** — Use the retrieved context to generate accurate answers

This pattern illustrates a fundamental concept in AI systems: **LLMs have knowledge cutoffs and can hallucinate**. By retrieving relevant context from a curated knowledge base before generating a response, we can provide accurate, up-to-date answers grounded in real documentation.

We use the OpenAI SDK (with OpenRouter as our provider) for LLM calls and embeddings, and Supabase (Postgres + pgvector) for vector storage — keeping the implementation transparent with no heavy frameworks.

# Installs and Imports

In [38]:
%%capture
!pip install openai python-dotenv supabase

`%%capture` hides the pip install output to keep the notebook clean.

In [39]:
# OpenAI SDK: provides a clean interface for calling LLMs and creating embeddings
# We'll use it with OpenRouter, which is OpenAI API-compatible
from openai import OpenAI

# Supabase client for vector database operations
from supabase import create_client

# Standard library
import os
import hashlib  # For generating content hashes to prevent duplicate document insertions

**OpenRouter** is a unified API that provides access to various LLMs through a single interface. It offers a generous free tier and affordable token usage, making it ideal for learning and experimentation. We also use it for embeddings via OpenAI's embedding models.

**Supabase** is an open-source Firebase alternative built on top of PostgreSQL. It provides a hosted Postgres database with the **pgvector** extension, which enables efficient vector similarity search 

# Setup your API Keys

## Step 1 — Get an OpenRouter API key

For this demo we will use an LLM and embeddings via OpenRouter, which requires an API key.

1. Go to https://openrouter.ai

2. Sign in (or create an account if you don't have one)

3. Once logged in, navigate to https://openrouter.ai/settings/keys

4. Click **Create Key**

5. Give the key a name, e.g. `colab-rag-workflow`

6. Copy the key immediately (you won't be able to see it again)

**Important: Treat this key like a password. Do not share it, paste it into notebooks, or commit it to GitHub.**

## Step 2 — Create a Supabase project and enable pgvector

We need a Supabase project to store our document embeddings.

1. Go to https://supabase.com

2. Sign in (or create a free account)

3. Click **New Project**

4. Give your project a name (e.g., `fasthtml-tutor`) and set a database password

5. Select a region close to you and click **Create new project**

6. Wait for the project to be provisioned (this takes about 2 minutes)

Once your project is ready, enable the pgvector extension:

1. In your project dashboard, go to **SQL Editor** (left sidebar)

2. Run the following SQL command:
   ```sql
   create extension if not exists vector;
   ```

3. Click **Run** to execute

This enables vector operations in your database, which we'll need for similarity search.

## Step 3 — Get your Supabase credentials

You'll need two values from your Supabase project:

1. In your project dashboard, go to **Settings** (gear icon) → **Data API**

2. Under "Project URL", copy the **URL** (looks like `https://xxxxx.supabase.co`)

3. Under "Project API keys", copy the **anon public** key

These credentials allow your code to connect to your Supabase database.

## Step 4 — Store your API keys

### If running in Google Colab:

1. On the left sidebar, click 🔑 **Secrets**

2. Add three new secrets:
   - Name: `OPENROUTER_API_KEY` → Value: your OpenRouter API key
   - Name: `SUPABASE_URL` → Value: your Supabase project URL
   - Name: `SUPABASE_KEY` → Value: your Supabase anon key

3. Toggle the switch to give notebook access to each secret

### If running locally:

1. Create a `.env` file in the project root:
   ```
   touch .env
   ```

2. Add the following (replace with your own values):
   ```
   OPENROUTER_API_KEY=your_openrouter_key_here
   SUPABASE_URL=https://xxxxx.supabase.co
   SUPABASE_KEY=your_supabase_anon_key_here
   ```

**Important: Never paste API keys directly into code cells.**

## Load the API Keys

### In Colab:

Uncomment and run the cell below if you're using Google Colab.

In [ ]:
# # Uncomment the lines below if running in Google Colab
# from google.colab import userdata
# os.environ["OPENROUTER_API_KEY"] = userdata.get("OPENROUTER_API_KEY")
# os.environ["SUPABASE_URL"] = userdata.get("SUPABASE_URL")
# os.environ["SUPABASE_KEY"] = userdata.get("SUPABASE_KEY")
# print("API keys loaded from Colab Secrets")

### Locally:

Run the cell below if you're running locally with a `.env` file.

In [40]:
# Load API keys from .env file if running locally
from dotenv import load_dotenv
load_dotenv()
print("Environment loaded from .env file")

Environment loaded from .env file


In [41]:
# Sanity check: verify all required credentials are available
openrouter_key = os.getenv("OPENROUTER_API_KEY")
supabase_url = os.getenv("SUPABASE_URL")
supabase_key = os.getenv("SUPABASE_KEY")

assert openrouter_key is not None, "OPENROUTER_API_KEY not found. Please check your setup."
assert supabase_url is not None, "SUPABASE_URL not found. Please check your setup."
assert supabase_key is not None, "SUPABASE_KEY not found. Please check your setup."

print("OPENROUTER_API_KEY present:", bool(openrouter_key))
print("SUPABASE_URL present:", bool(supabase_url))
print("SUPABASE_KEY present:", bool(supabase_key))

OPENROUTER_API_KEY present: True
SUPABASE_URL present: True
SUPABASE_KEY present: True


# Initialize the Clients

We need two clients for our RAG workflow:

1. **OpenAI client** — Points to OpenRouter for LLM calls and embeddings
2. **Supabase client** — Connects to our Postgres database for vector storage

We'll also define the models we'll use:
- **Chat model**: `openai/gpt-4o-mini` for generating responses
- **Embedding model**: `openai/text-embedding-3-small` for creating vector embeddings

In [42]:
# Initialize the OpenAI client pointing to OpenRouter
openai_client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY")
)

# Initialize the Supabase client
supabase = create_client(
    os.getenv("SUPABASE_URL"),
    os.getenv("SUPABASE_KEY")
)

# Models to use
CHAT_MODEL = "openai/gpt-4o-mini"
EMBEDDING_MODEL = "openai/text-embedding-3-small"

print(f"OpenAI client initialized. Using chat model: {CHAT_MODEL}")
print(f"Embedding model: {EMBEDDING_MODEL}")
print(f"Supabase client initialized for: {os.getenv('SUPABASE_URL')}")

OpenAI client initialized. Using chat model: openai/gpt-4o-mini
Embedding model: openai/text-embedding-3-small
Supabase client initialized for: https://xvckovfrkufaulixokrr.supabase.co


# Step 1: Document Loading

The first step in building a RAG system is loading the source documents. In our case, we have a single text file containing the FastHTML documentation.

In [43]:
# Load the FastHTML documentation
with open("FastHTML.txt", "r", encoding="utf-8") as f:
    document = f.read()

print(f"Loaded document with {len(document)} characters")
document[:200]

Loaded document with 403766 characters


'### Minimal FastHTML Application Example\n\nSource: https://www.fastht.ml/docs/ref/concise_guide.html\n\nDemonstrates a basic FastHTML application setup. It includes defining an app instance, a route with'

# Step 2: Chunking

Large documents need to be split into smaller pieces (chunks) for two reasons:

1. **Embedding models have token limits** — Most embedding models can only process a limited amount of text at once
2. **Retrieval precision** — Smaller chunks allow us to retrieve more relevant, focused context

## Chunking Strategy

We'll implement a simple **recursive character text splitter** to demonstrate how chunking works:
- Split the text into chunks of approximately 1000 characters
- Include 200 characters of overlap between chunks to preserve context across boundaries

The overlap ensures that if important information spans a chunk boundary, it will appear in at least one complete chunk.

**Note:** Since our FastHTML documentation is in markdown format, a better choice might be to use a markdown-aware splitter such as `MarkdownTextSplitter` from the `langchain-text-splitters` package. These splitters respect markdown structure (headers, code blocks, etc.) and can preserve header hierarchy as metadata. Here, we implement a simple character-based splitter to illustrate the core concepts.

In [44]:
def chunk_text(text: str, chunk_size: int = 1000, overlap: int = 200) -> list[str]:
    """
    Split text into overlapping chunks.
    
    Args:
        text: The text to split
        chunk_size: Target size for each chunk (in characters)
        overlap: Number of characters to overlap between chunks
        
    Returns:
        List of text chunks
    """
    chunks = []
    start = 0
    
    while start < len(text):
        # Get the chunk
        end = start + chunk_size
        chunk = text[start:end]
        
        # Try to break at a natural boundary (newline or period)
        if end < len(text):
            # Look for a good break point near the end
            last_newline = chunk.rfind('\n')
            last_period = chunk.rfind('. ')
            
            # Use the later of the two break points, if found in the last 20% of the chunk
            break_point = max(last_newline, last_period)
            if break_point > chunk_size * 0.8:
                chunk = text[start:start + break_point + 1]
                end = start + break_point + 1
        
        chunks.append(chunk.strip())
        
        # Move start position, accounting for overlap
        start = end - overlap
    
    return chunks

In [45]:
# Split the document into chunks
chunks = chunk_text(document, chunk_size=1000, overlap=200)

print(f"Created {len(chunks)} chunks")
print(f"\nExample chunk (chunk 0):\n{chunks[0][:500]}...")
print(f"\nChunk sizes: min={min(len(c) for c in chunks)}, max={max(len(c) for c in chunks)}, avg={sum(len(c) for c in chunks)//len(chunks)}")

Created 527 chunks

Example chunk (chunk 0):
### Minimal FastHTML Application Example

Source: https://www.fastht.ml/docs/ref/concise_guide.html

Demonstrates a basic FastHTML application setup. It includes defining an app instance, a route with type-constrained parameters, and serving the application. The example highlights FastHTML's approach to routing, HTML generation using FastTags, and automatic server startup.

```python
# Meta-package with all key symbols from FastHTML and Starlette. Import it like this at the start of every FastHT...

Chunk sizes: min=619, max=1000, avg=964


# Step 3: Create Embeddings

**Embeddings** are numerical representations of text that capture semantic meaning. Similar texts will have similar embeddings (vectors that are close together in high-dimensional space).

We use OpenAI's `text-embedding-3-small` model, which produces 1536-dimensional vectors. 

## How Embeddings Work

1. Text goes into the embedding model
2. The model outputs a vector of 1536 floating-point numbers
3. These numbers encode the semantic meaning of the text
4. Similar meanings → similar vectors → can be found via similarity search

In [46]:
def get_embedding(text: str) -> list[float]:
    """
    Get the embedding vector for a piece of text.
    
    Args:
        text: The text to embed
        
    Returns:
        A list of floats representing the embedding vector
    """
    response = openai_client.embeddings.create(
        model=EMBEDDING_MODEL,
        input=text
    )
    return response.data[0].embedding

**Pedagogy:** Always understand what you are working with! Explore `response` directly to understand it's structure

In [47]:
response = openai_client.embeddings.create(
        model=EMBEDDING_MODEL,
        input="What is FastHTML?"
    )


In [48]:
response.model_dump()


{'data': [{'embedding': [-0.07315411418676376,
    -0.005316810682415962,
    0.019212059676647186,
    0.014737483114004135,
    0.0064777289517223835,
    -0.04974063113331795,
    -0.03868426755070686,
    0.04136380925774574,
    0.02791406773030758,
    0.00781750027090311,
    0.048700034618377686,
    0.004559124354273081,
    -0.007570357993245125,
    -0.03374142199754715,
    0.019589276984333992,
    0.01966732181608677,
    0.004282715264707804,
    -0.05556798726320267,
    -0.005593219771981239,
    -0.01969333551824093,
    0.009983247146010399,
    -0.05660858750343323,
    0.007375245448201895,
    -0.010425501503050327,
    -0.049532514065504074,
    0.014685453847050667,
    -0.029058726504445076,
    -0.012318090535700321,
    0.016025224700570107,
    0.005765568930655718,
    0.0051412093453109264,
    -0.04016712307929993,
    0.06852344423532486,
    0.019862433895468712,
    -0.0036648595705628395,
    0.027185648679733276,
    0.028902636840939522,
    0.04672

In [49]:
# Test the embedding function with a sample text
sample_embedding = get_embedding("What is FastHTML?")
print(f"Embedding dimension: {len(sample_embedding)}")
print(f"First 10 values: {sample_embedding[:10]}")

Embedding dimension: 1536
First 10 values: [-0.07315411418676376, -0.005316810682415962, 0.019212059676647186, 0.014737483114004135, 0.0064777289517223835, -0.04974063113331795, -0.03868426755070686, 0.04136380925774574, 0.02791406773030758, 0.00781750027090311]


In [50]:
# Generate embeddings for all chunks
# This may take a minute depending on the number of chunks
print(f"Generating embeddings for {len(chunks)} chunks...")

embeddings = []
for i, chunk in enumerate(chunks):
    embedding = get_embedding(chunk)
    embeddings.append(embedding)
    if (i + 1) % 50 == 0:
        print(f"  Processed {i + 1}/{len(chunks)} chunks")

print(f"Done! Generated {len(embeddings)} embeddings")

Generating embeddings for 527 chunks...
  Processed 50/527 chunks
  Processed 100/527 chunks
  Processed 150/527 chunks
  Processed 200/527 chunks
  Processed 250/527 chunks
  Processed 300/527 chunks
  Processed 350/527 chunks
  Processed 400/527 chunks
  Processed 450/527 chunks
  Processed 500/527 chunks
Done! Generated 527 embeddings


# Step 4: Store in Supabase

Now we store our chunks and their embeddings in Supabase so we can search them later.

## Create the Table

First, create a table to store the documents. Go to your Supabase dashboard → **SQL Editor** and run the following SQL:

```sql
create table if not exists documents (
  id bigserial primary key,
  content text not null,
  embedding vector(1536),
  content_hash text unique
);
```

This creates a table with:
- `id`: Auto-incrementing primary key
- `content`: The text chunk
- `embedding`: A 1536-dimensional vector (matching our embedding model's output)
- `content_hash`: A unique hash of the content to prevent duplicate insertions

The `content_hash` column acts as a simple **record manager** — if you re-run the notebook, documents with the same content will be updated rather than duplicated.

In [ ]:
def get_content_hash(text: str) -> str:
    """Generate a hash of the content to use as a unique identifier."""
    return hashlib.md5(text.encode()).hexdigest() # creates a hash as a 32-character hex string

# Insert all chunks and embeddings into Supabase using upsert
# This prevents duplicates if you re-run the notebook
print(f"Upserting {len(chunks)} documents into Supabase...")

for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
    content_hash = get_content_hash(chunk)
    
    supabase.table("documents").upsert({
        "content": chunk,
        "embedding": embedding,
        "content_hash": content_hash
    }, on_conflict="content_hash").execute()
    
    if (i + 1) % 50 == 0:
        print(f"  Processed {i + 1}/{len(chunks)} documents")

print(f"Done! Upserted {len(chunks)} documents into Supabase")

Upserting 527 documents into Supabase...
  Processed 50/527 documents
  Processed 100/527 documents
  Processed 150/527 documents
  Processed 200/527 documents
  Processed 250/527 documents
  Processed 300/527 documents
  Processed 350/527 documents
  Processed 400/527 documents
  Processed 450/527 documents
  Processed 500/527 documents
Done! Upserted 527 documents into Supabase


# Step 5: Retrieval

Now we can search our vector database to find chunks relevant to a user's question.

## Create a Search Function in Supabase

To perform similarity search, we need a database function. Go to **SQL Editor** in Supabase and run:

```sql
create or replace function match_documents (
  query_embedding vector(1536),
  match_count int default 5
)
returns table (
  id bigint,
  content text,
  similarity float
)
language plpgsql
as $$
begin
  return query
  select
    documents.id,
    documents.content,
    1 - (documents.embedding <=> query_embedding) as similarity
  from documents
  order by documents.embedding <=> query_embedding
  limit match_count;
end;
$$;
```


This function:
- Takes a query embedding and returns the most similar documents
- Uses cosine distance (`<=>`) to measure similarity
- Returns results ordered by similarity (highest first)
```
"""
 <=> is cosine distance operator (0 = identical, 2 = opposite)
 %s::vector casts the Python list parameter to a Postgres vector type
 1 - (embedding <=> %s::vector) as similarity - Converts distance to similarity (1 = identical, -1 = opposite)
"""
```



In [53]:
def search_documents(query: str, match_count: int = 5) -> list[dict]:
    """
    Search for documents similar to the query.
    
    Args:
        query: The search query
        match_count: Number of results to return
        
    Returns:
        List of matching documents with their similarity scores
    """
    # Get embedding for the query
    query_embedding = get_embedding(query)
    
    # Call the Supabase function
    result = supabase.rpc(
        "match_documents",
        {"query_embedding": query_embedding, "match_count": match_count}
    ).execute()
    
    return result.data

In [54]:
# Test the search function
results = search_documents("How do I create a route in FastHTML?", match_count=3)

print(f"Found {len(results)} matching documents:\n")
for i, doc in enumerate(results):
    print(f"--- Result {i+1} (similarity: {doc['similarity']:.4f}) ---")
    print(doc['content'][:300] + "...")
    print()

Found 3 matching documents:

--- Result 1 (similarity: 0.6302) ---
----------------

### Initialize FastHTML App and Router

Source: https://www.fastht.ml/docs/tutorials/jupyter_and_fasthtml.html

Sets up a new FastHTML application instance and its associated router. The `pico=True` argument suggests enabling a lightweight mode for the application.

```python
app, ...

--- Result 2 (similarity: 0.6176) ---
tps://www.fastht.ml/docs/apilist.txt

Initializes the FastHTML application, supporting various configurations including debugging, routing, middleware, title, exception handlers, and lifecycle events. It also integrates with HTMX and provides options for session management.

```python
class FastHTML...

--- Result 3 (similarity: 0.6076) ---
als/by_example.html

Demonstrates how to capture path parameters in FastHTML routes. The captured parameter 'nm' is used in the response string. It also shows how to test this route using TestClient.

```python
@app.get('/user/{nm}')
def _(nm:str)

# Step 6: Generation

Now we combine everything into a complete RAG pipeline:
1. Take the user's question
2. Retrieve relevant documents from our vector database
3. Include those documents as context in the prompt
4. Generate an answer using the LLM

RAG: we augment the LLM's knowledge with retrieved context.

In [55]:
def ask_fasthtml_tutor(question: str, num_docs: int = 5) -> str:
    """
    Ask the FastHTML tutor a question.
    
    Args:
        question: The user's question about FastHTML
        num_docs: Number of documents to retrieve for context
        
    Returns:
        The tutor's response
    """
    # Step 1: Retrieve relevant documents
    docs = search_documents(question, match_count=num_docs)
    
    # Step 2: Build context from retrieved documents
    context = "\n\n---\n\n".join([doc["content"] for doc in docs])
    
    # Step 3: Create the prompt with context
    system_prompt = """You are a helpful FastHTML tutor. Answer questions about FastHTML based on the provided documentation context. 

If the context doesn't contain enough information to answer the question, say so honestly. 
Always be accurate and cite specific examples from the documentation when possible."""

    user_prompt = f"""Context from FastHTML documentation:

{context}

---

Question: {question}

Please provide a clear, helpful answer based on the documentation above."""

    # Step 4: Generate response
    response = openai_client.chat.completions.create(
        model=CHAT_MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.7
    )
    
    return response.choices[0].message.content

In [36]:
# Test the complete RAG pipeline
question = "How do I create a route in FastHTML?"
answer = ask_fasthtml_tutor(question)

print(f"Question: {question}\n")
print(f"Answer:\n{answer}")

Question: How do I create a route in FastHTML?

Answer:
To create a route in FastHTML, you can use the `@app.get` decorator to define a route that responds to HTTP GET requests. Here's a simple example based on the documentation:

```python
@app.get('/user/{nm}')
def _(nm: str):
    return f"Good day to you, {nm}!"
```

In this example, the route is defined for the path `/user/{nm}`, where `{nm}` is a path parameter that can be captured and used in the response. When a user accesses this route with a specific name (for example, `/user/jph`), the response will be "Good day to you, jph!".

Additionally, you can define routes using the `app.route` method, which allows you to specify the HTTP method by naming the function appropriately. However, the documentation highlights the use of decorators like `@app.get` for simplicity when defining routes for specific HTTP methods.

For testing this route, you can utilize the `TestClient` as shown in the documentation:

```python
cli.get('/user/jph

### Here is another question:

In [56]:
# Try your own question!
my_question = "What are FastTags in FastHTML?"
answer = ask_fasthtml_tutor(my_question)

print(f"Question: {my_question}\n")
print(f"Answer:\n{answer}")

Question: What are FastTags in FastHTML?

Answer:
FastTags in FastHTML are components that allow you to create basic HTML tags using FastHTML's m-expression syntax. In this context, positional parameters become the children of the tags, while named parameters are treated as attributes of those tags. 

For example, you can create a `Title`, `H1`, and `P` tag as follows:

```python
tags = Title("FastHTML"), H1("My web app"), P(f"Let's do this!", cls="myclass")
```

In this example:
- `Title("FastHTML")` creates a title tag with the text "FastHTML".
- `H1("My web app")` creates a header tag with the text "My web app".
- `P(f"Let's do this!", cls="myclass")` creates a paragraph tag with some text and assigns a class attribute "myclass".

FastTags are part of the FastHTML framework's approach to rendering HTML, allowing for an intuitive and structured way to build HTML content programmatically.
